In [1]:
%cd ..

/Users/chan/boostcamp-project/level2-nlp-generationfornlp-nlp-06-lv3


/Users/chan/boostcamp-project/level2-nlp-generationfornlp-nlp-06-lv3/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
import os

import pandas as pd
import wikipediaapi as wk
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from tqdm import tqdm

from prompts import load_template
from utils import create_experiment_file_name

In [ ]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
WIKI_USER_AGENT = os.getenv("WIKI_USER_AGENT")

DATA_DIR = "data"
RESULT_DIR = "experiments"
TARGET_DATA = "train_v2.0.1.csv"
EXP_VERSION = "v1.0.2"  # 컨벤션에 따라 새롭게 실험할 경우 버저닝 정보를 기록해주세요.

PARAGRAPH = "paragraph"
QUESTION = "question"
CHOICES = "choices"
QUESTION_PLUS = "question_plus"

In [ ]:
datasets = pd.read_csv(os.path.join(DATA_DIR, TARGET_DATA))
datasets.head()

,id,paragraph,question,choices,answer,question_plus
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?,"['ㄱ, ㄴ', 'ㄱ, ㄷ', 'ㄴ, ㄹ', 'ㄷ, ㄹ']",2,ㄱ.기 사환국으로 정권을 장악하였다 .ㄴ.인 조반정을 주도 하여 집권세력이 되었다 ...
1,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,(가) 지역에 대한 설명으로 옳은 것은?,"['이곳에 대장도감을 설치하여 재조대장경을 만들었다.', '지눌이 이곳에서 수선사 ...",4,NaN
2,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,밑줄 친 ‘그’에 대한 설명으로 옳은 것은?,"['살수에서 수의 군대를 물리쳤다 .', '김춘추 의 신라 왕위 계승을 지원하였다 ...",2,NaN
3,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...",(가) 인물이 추진한 정책으로 옳지 않은 것은?,"['사창제를 실시하였다 .', '대전회통을 편찬하였다 .', '비변사의 기능을 강화...",3,NaN
4,generation-for-nlp-431,(가)의 사신 고제덕 등이 일본에 와서 왕이 보낸 국서를 전하였다. 그 국서에 이르...,(가) 국가에 대한 설명으로 옳은 것은?,"['나 당연합군의 공격으로 멸망하였다 .', '9주 5소경의 지방 행정 구역을 두었...",3,NaN


In [ ]:
prompt = PromptTemplate.from_template(
    template=load_template(file_name="wikipedia_search_keyword.txt", template_type="keyword_extraction")
)

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini", temperature=0, max_tokens=None, timeout=None, max_retries=2, api_key=OPENAI_API_KEY
)

keyword_extractor = prompt | llm

In [ ]:
def extract_keyword(data: pd.Series) -> list[str]:
    """
    문제 해결에 필요한 키워드 5개를 추출하고 결과를 배열로 반환하는 함수

    Args:
        data (pd.Series): 제공된 데이터셋의 한 행(row), 문제 하나

    Returns:
        list[str]: 추출된 5개 키워드
    """
    response = keyword_extractor.invoke(
        {
            "paragraph": data[PARAGRAPH],
            "question": data[QUESTION],
            "question_plus": data[QUESTION_PLUS],
            "choices": data[CHOICES],
        },
    )
    response_split = response.content.split(",")
    keywords = [keyword.strip() for keyword in response_split]
    return keywords


tqdm.pandas()
datasets["keywords"] = datasets.progress_apply(extract_keyword, axis=1)
datasets.head()

100%|██████████| 1627/1627 [30:57<00:00,  1.14s/it]  


,id,paragraph,question,choices,answer,question_plus,keywords
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?,"['ㄱ, ㄴ', 'ㄱ, ㄷ', 'ㄴ, ㄹ', 'ㄷ, ㄹ']",2,ㄱ.기 사환국으로 정권을 장악하였다 .ㄴ.인 조반정을 주도 하여 집권세력이 되었다 ...,"[상소, 송준길, 상복, 정체, 붕당]"
1,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,(가) 지역에 대한 설명으로 옳은 것은?,"['이곳에 대장도감을 설치하여 재조대장경을 만들었다.', '지눌이 이곳에서 수선사 ...",4,NaN,"[삼한, 수덕, 지맥, 고려사, 봉기]"
2,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,밑줄 친 ‘그’에 대한 설명으로 옳은 것은?,"['살수에서 수의 군대를 물리쳤다 .', '김춘추 의 신라 왕위 계승을 지원하였다 ...",2,NaN,"[소정방, 김인문, 신라군, 황산 전투, 백제]"
3,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...",(가) 인물이 추진한 정책으로 옳지 않은 것은?,"['사창제를 실시하였다 .', '대전회통을 편찬하였다 .', '비변사의 기능을 강화...",3,NaN,"[선비, 대궐, 서원, 정책, 한성부]"
4,generation-for-nlp-431,(가)의 사신 고제덕 등이 일본에 와서 왕이 보낸 국서를 전하였다. 그 국서에 이르...,(가) 국가에 대한 설명으로 옳은 것은?,"['나 당연합군의 공격으로 멸망하였다 .', '9주 5소경의 지방 행정 구역을 두었...",3,NaN,"[고구려, 대무예, 국서, 부여, 중앙 행정]"


In [ ]:
result_file_name = create_experiment_file_name(target_file=TARGET_DATA, version=EXP_VERSION)
result_file_path = os.path.join(DATA_DIR, RESULT_DIR, result_file_name)
datasets.to_csv(result_file_path, index=False)

추출된 키워드가 위키피디아에 존재하는지 여부 확인

- 키워드 추출 결과가 유효한지 확인하기 위한 작업입니다.


In [ ]:
wiki = wk.Wikipedia(user_agent=WIKI_USER_AGENT, language="ko")

In [ ]:
# 키워드를 유용하게 다루기 위해 키워드 별로 칼럼으로 분리
keywords_expanded = pd.DataFrame(datasets["keywords"].tolist(), index=datasets.index)
keywords_expanded.columns = [f"keyword_{i+1}" for i in range(keywords_expanded.shape[1])]
new_datasets = pd.concat([datasets.drop(columns=["keywords"]), keywords_expanded], axis=1)

new_datasets.head()

,id,paragraph,question,choices,answer,question_plus,keyword_1,keyword_2,keyword_3,keyword_4,keyword_5
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?,"['ㄱ, ㄴ', 'ㄱ, ㄷ', 'ㄴ, ㄹ', 'ㄷ, ㄹ']",2,ㄱ.기 사환국으로 정권을 장악하였다 .ㄴ.인 조반정을 주도 하여 집권세력이 되었다 ...,상소,송준길,상복,정체,붕당
1,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,(가) 지역에 대한 설명으로 옳은 것은?,"['이곳에 대장도감을 설치하여 재조대장경을 만들었다.', '지눌이 이곳에서 수선사 ...",4,NaN,삼한,수덕,지맥,고려사,봉기
2,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,밑줄 친 ‘그’에 대한 설명으로 옳은 것은?,"['살수에서 수의 군대를 물리쳤다 .', '김춘추 의 신라 왕위 계승을 지원하였다 ...",2,NaN,소정방,김인문,신라군,황산 전투,백제
3,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...",(가) 인물이 추진한 정책으로 옳지 않은 것은?,"['사창제를 실시하였다 .', '대전회통을 편찬하였다 .', '비변사의 기능을 강화...",3,NaN,선비,대궐,서원,정책,한성부
4,generation-for-nlp-431,(가)의 사신 고제덕 등이 일본에 와서 왕이 보낸 국서를 전하였다. 그 국서에 이르...,(가) 국가에 대한 설명으로 옳은 것은?,"['나 당연합군의 공격으로 멸망하였다 .', '9주 5소경의 지방 행정 구역을 두었...",3,NaN,고구려,대무예,국서,부여,중앙 행정


In [11]:
# 위키피디아 페이지가 있는지 키워드 별로 확인
for col in keywords_expanded.columns:
    new_datasets[f"{col}_exists"] = new_datasets[col].apply(lambda x: wiki.page(x).exists())

new_datasets.head()

,id,paragraph,question,choices,answer,question_plus,keyword_1,keyword_2,keyword_3,keyword_4,keyword_5,keyword_1_exists,keyword_2_exists,keyword_3_exists,keyword_4_exists,keyword_5_exists
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?,"['ㄱ, ㄴ', 'ㄱ, ㄷ', 'ㄴ, ㄹ', 'ㄷ, ㄹ']",2,ㄱ.기 사환국으로 정권을 장악하였다 .ㄴ.인 조반정을 주도 하여 집권세력이 되었다 ...,상소,송준길,상복,정체,붕당,True,True,True,True,True
1,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,(가) 지역에 대한 설명으로 옳은 것은?,"['이곳에 대장도감을 설치하여 재조대장경을 만들었다.', '지눌이 이곳에서 수선사 ...",4,NaN,삼한,수덕,지맥,고려사,봉기,True,True,False,True,True
2,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,밑줄 친 ‘그’에 대한 설명으로 옳은 것은?,"['살수에서 수의 군대를 물리쳤다 .', '김춘추 의 신라 왕위 계승을 지원하였다 ...",2,NaN,소정방,김인문,신라군,황산 전투,백제,True,True,False,True,True
3,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...",(가) 인물이 추진한 정책으로 옳지 않은 것은?,"['사창제를 실시하였다 .', '대전회통을 편찬하였다 .', '비변사의 기능을 강화...",3,NaN,선비,대궐,서원,정책,한성부,True,True,True,True,True
4,generation-for-nlp-431,(가)의 사신 고제덕 등이 일본에 와서 왕이 보낸 국서를 전하였다. 그 국서에 이르...,(가) 국가에 대한 설명으로 옳은 것은?,"['나 당연합군의 공격으로 멸망하였다 .', '9주 5소경의 지방 행정 구역을 두었...",3,NaN,고구려,대무예,국서,부여,중앙 행정,True,True,True,True,False


In [ ]:
# _exists 열만 추출
exists_columns = [col for col in new_datasets.columns if col.endswith("_exists")]

true_count = new_datasets[exists_columns].sum().sum()  # True 값의 총합
false_count = (new_datasets[exists_columns] == False).sum().sum()  # False 값의 총합

true_count, false_count

(5511, 2624)

In [ ]:
result_file_path = os.path.join(DATA_DIR, RESULT_DIR, "train_exp_v1.0.2.csv")
new_datasets.to_csv()

',id,paragraph,question,choices,answer,question_plus,keyword_1,keyword_2,keyword_3,keyword_4,keyword_5,keyword_1_exists,keyword_2_exists,keyword_3_exists,keyword_4_exists,keyword_5_exists\n0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服) 절차에 대하여 논한 것이 신과는 큰 차이가 있었습니다 . 장자를 위하여 3년을 입는 까닭은 위로 ‘정체(正體)’가 되기 때문이고 또 전 중(傳重: 조상의 제사나 가문의 법통을 전함)하기 때문입니다 . …(중략) … 무엇보다 중요한 것은 할아버지와 아버지의 뒤를 이은 ‘정체’이지, 꼭 첫째이기 때문에 참 최 3년 복을 입는 것은 아닙니다 .”라고 하였다 .－현종실록 －",상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?,"[\'ㄱ, ㄴ\', \'ㄱ, ㄷ\', \'ㄴ, ㄹ\', \'ㄷ, ㄹ\']",2,ㄱ.기 사환국으로 정권을 장악하였다 .ㄴ.인 조반정을 주도 하여 집권세력이 되었다 .ㄷ.정조 시기에 탕평 정치의 한 축을 이루었다 .ㄹ.이 이와 성혼의 문인을 중심으로 형성되었다.,상소,송준길,상복,정체,붕당,True,True,True,True,True\n1,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순조로워 우리나라 지맥의 뿌리가 되니 대업을 만대에 전할 땅이다. 왕은 춘하 추동네 계절의 중간달에 그곳에 가 100일 이상 머물러서 나라를 안녕케 하라. － 고려사－,(가) 지역에 대한 설명으로 옳은 것은?,"[\'이곳에 대장도감을 설치하여 재조대장경을 만들었다.\', \'지눌이 이곳에서 수선사 결사운동을 펼쳤다.\', \'망이 ․망소이가 이곳에서 봉기하였다.\', \'몽골이 이곳에 동녕부를 두었다.\']",4,,